In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load data

In [2]:
# get processed dataset
all_students_features = pd.read_csv('all_students_features_0904.csv')

In [3]:
print(all_students_features.shape)
all_students_features.head()

(2891563, 16)


,Unnamed: 0,employee_id_hash,is_greek,is_athlete,academic_career,academic_plan1,hd_notify_date,day_idx,positives_identified_on_this_day,infected_on_this_day,previous_infection,week_idx,biweek_idx,class_positivity_in_previous_week,underlying_positives_on_this_day,underlying_positives_on_this_day_forward_backward
0,0,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-26,23,0,0,0,0,0.0,296.0,296.0
1,1,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-27,43,0,0,0,0,0.0,315.0,338.0
2,2,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-28,57,0,0,0,0,0.0,298.0,364.0
3,3,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-29,55,0,0,0,0,0.0,270.0,393.0
4,4,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-30,39,0,0,0,0,0.0,219.0,397.0


In [4]:
# get dataframe for regression

x = all_students_features[['employee_id_hash', 'is_greek', 'is_athlete', 'academic_career',
                          'day_idx', 
                          'week_idx', 'class_positivity_in_previous_week', 
                            'underlying_positives_on_this_day', 
                           'underlying_positives_on_this_day_forward_backward',
                          'infected_on_this_day']].dropna(subset=['class_positivity_in_previous_week', 'underlying_positives_on_this_day'])

x.drop(x[x.academic_career=='EE'].index, inplace = True)
x.reset_index(drop=True, inplace=True)

# Descriptive statistics

In [5]:
# number of person-days 
x.shape

(2839592, 10)

In [6]:
# size of population studied
len(x['employee_id_hash'].unique())

27802

In [7]:
# number of negative and positive person-days
x['infected_on_this_day'].value_counts()

0    2835535
1       4057
Name: infected_on_this_day, dtype: int64

In [8]:
# average class-positivity value across the person-days
x['class_positivity_in_previous_week'].mean()

0.014772664859709676

In [9]:
# descriptive stats for each academic career subgroup
for ac in ['UG_G', 'UG_A', 'UG', 'GM', 'GR', 'LA', 'VM']:

    x_sub = x.query('academic_career == @ac')
    
    print('====Academic career: ', ac, '====')

    print('number of person-days: ', x_sub.shape[0])
    print('average class positivity: ', x_sub['class_positivity_in_previous_week'].mean())
    print('population size: ', len(x_sub['employee_id_hash'].unique()))
    print('number of positive and negative person-days: ', list(x_sub['infected_on_this_day'].value_counts()))


====Academic career:  UG_G ====
number of person-days:  314434
average class positivity:  0.014861124703277573
population size:  3282
number of positive and negative person-days:  [312830, 1604]
====Academic career:  UG_A ====
number of person-days:  104420
average class positivity:  0.014406676687334603
population size:  1044
number of positive and negative person-days:  [104129, 291]
====Academic career:  UG ====
number of person-days:  1215889
average class positivity:  0.014861504310610202
population size:  11835
number of positive and negative person-days:  [1214531, 1358]
====Academic career:  GM ====
number of person-days:  177586
average class positivity:  0.0149383559924807
population size:  1716
number of positive and negative person-days:  [177485, 101]
====Academic career:  GR ====
number of person-days:  880845
average class positivity:  0.014670668993725645
population size:  8498
number of positive and negative person-days:  [880423, 422]
====Academic career:  LA ====
num

# Run logistic regression

In [10]:
model_sm_fb = smf.logit(formula='infected_on_this_day ~ C(academic_career, Treatment(reference = "UG")) + class_positivity_in_previous_week + underlying_positives_on_this_day_forward_backward', 
                 data=x)
                     
res_fb = model_sm_fb.fit(maxiter = 100, method = 'cg')
res_fb.summary()

Optimization terminated successfully.
         Current function value: 0.010279
         Iterations: 59
         Function evaluations: 195
         Gradient evaluations: 195


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                            
================================================================================
Dep. Variable:     infected_on_this_day   No. Observations:              2839592
Model:                            Logit   Df Residuals:                  2839583
Method:                             MLE   Df Model:                            8
Date:                  Sat, 22 Oct 2022   Pseudo R-squ.:                 0.04708
Time:                          15:30:22   Log-Likelihood:                -29189.
converged:                         True   LL-Null:                       -30631.
Covariance Type:              nonrobust   LLR p-value:                     0.000
=========================================================================================================================
                                                            coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                                -6.8186      0.029   -232.564      0.000      -6.876      -6.761
C(academic_career, Treatment(reference="UG"))[T.GM]      -0.7955      0.108     -7.395      0.000      -1.006      -0.585
C(academic_career, Treatment(reference="UG"))[T.GR]      -0.8422      0.055    -15.351      0.000      -0.950      -0.735
C(academic_career, Treatment(reference="UG"))[T.LA]       0.5106      0.079      6.473      0.000       0.356       0.665
C(academic_career, Treatment(reference="UG"))[T.UG_A]     0.9228      0.064     14.511      0.000       0.798       1.047
C(academic_career, Treatment(reference="UG"))[T.UG_G]     1.4935      0.037     40.741      0.000       1.422       1.565
C(academic_career, Treatment(reference="UG"))[T.VM]       0.1037      0.127      0.819      0.413      -0.145       0.352
class_positivity_in_previous_week                        -0.1162      0.303     -0.384      0.701      -0.710       0.477
underlying_positives_on_this_day_forward_backward         0.0006      0.000      4.544      0.000       0.000       0.001
=========================================================================================================================
"""

In [20]:
res_fb.get_margeff().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects        
=======================================
Dep. Variable:     infected_on_this_day
Method:                            dydx
At:                             overall
=========================================================================================================================
                                                           dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
C(academic_career, Treatment(reference="UG"))[T.GM]      -0.0011      0.000     -7.346      0.000      -0.001      -0.001
C(academic_career, Treatment(reference="UG"))[T.GR]      -0.0012   8.08e-05    -14.930      0.000      -0.001      -0.001
C(academic_career, Treatment(reference="UG"))[T.LA]       0.0007      0.000      6.441      0.000       0.001       0.001
C(academic_career, Treatment(reference="UG"))[T.UG_A]     0.0013   9.33e-05     14.156      0.000       0.001       0.002
C(academic_career, Treatment(reference="UG"))[T.UG_G]     0.0021   6.21e-05     34.440      0.000       0.002       0.002
C(academic_career, Treatment(reference="UG"))[T.VM]       0.0001      0.000      0.819      0.413      -0.000       0.001
class_positivity_in_previous_week                        -0.0002      0.000     -0.384      0.701      -0.001       0.001
underlying_positives_on_this_day_forward_backward      9.106e-07   2.01e-07      4.533      0.000    5.17e-07     1.3e-06
=========================================================================================================================
"""

In [19]:
res_fb.get_margeff().summary_frame()

,dy/dx,Std. Err.,z,Pr(>|z|),Conf. Int. Low,Cont. Int. Hi.
"C(academic_career, Treatment(reference=""UG""))[T.GM]",-1.138851e-03,1.550290e-04,-7.346050,2.041504e-13,-1.442702e-03,-0.000835
"C(academic_career, Treatment(reference=""UG""))[T.GR]",-1.205707e-03,8.075541e-05,-14.930353,2.091574e-50,-1.363984e-03,-0.001047
"C(academic_career, Treatment(reference=""UG""))[T.LA]",7.309342e-04,1.134873e-04,6.440667,1.189496e-10,5.085031e-04,0.000953
"C(academic_career, Treatment(reference=""UG""))[T.UG_A]",1.321194e-03,9.333005e-05,14.156151,1.711140e-45,1.138271e-03,0.001504
"C(academic_career, Treatment(reference=""UG""))[T.UG_G]",2.138196e-03,6.208538e-05,34.439602,6.444267e-260,2.016511e-03,0.002260
"C(academic_career, Treatment(reference=""UG""))[T.VM]",1.484467e-04,1.813533e-04,0.818550,4.130432e-01,-2.069992e-04,0.000504
class_positivity_in_previous_week,-1.663075e-04,4.335126e-04,-0.383628,7.012543e-01,-1.015977e-03,0.000683
underlying_positives_on_this_day_forward_backward,9.106398e-07,2.008891e-07,4.533048,5.813866e-06,5.169044e-07,0.000001
